In [1]:
import numpy as np
import pandas as pd
import gc


In [2]:
seasons = list(range(2011,2020))

df = None

for s in seasons:
    path = './output/{}_boxscores.csv'.format(str(s))
    _df = pd.read_csv(path)
    
    if df is not None:
        df = pd.concat([df,_df],axis=0)
    else:
        df = _df
        
print(len(df))

df = df.drop(columns=['away_ranking','home_ranking'])


102617


In [3]:
all_teams = list(set(list(df['winning_name'].values)+list(df['losing_name'].values)))
print(all_teams[:15])

teams = []
for t in all_teams:
    if '\n\t\t\t' not in t:
        teams.append(t)
        
print(len(teams))
print(teams[:15])

['Clarion\n\t\t\t', 'Lindenwood\n\t\t\t', 'Central Baptist\n\t\t\t', 'Johnson &amp; Wales (NC)\n\t\t\t', 'Penn State-Beaver\n\t\t\t', 'Loyola (MD)', 'Howard', 'Ave Maria\n\t\t\t', 'San Francisco State\n\t\t\t', 'Texas Southern', 'NJIT', 'Lake Erie\n\t\t\t', 'Evergreen State\n\t\t\t', 'Albertus Magnus\n\t\t\t', 'Columbia International\n\t\t\t']
355
['Loyola (MD)', 'Howard', 'Texas Southern', 'NJIT', 'Davidson', 'South Dakota', 'Missouri State', 'Tennessee State', 'Air Force', 'California Baptist', 'Loyola (IL)', 'Charleston Southern', 'Youngstown State', 'Iowa State', 'Colgate']


In [4]:
def add_season(df):
    # datetime
    df['date'] = pd.to_datetime(df['date'])
    
    df['month'] = df['date'].copy().dt.month.astype(int)
    df['year'] = df['date'].copy().dt.year.astype(int)
    df['season'] = df['year'].copy()
    df['season'] = np.where(df['month']>9, df['season']+1, df['season'])
    
    df = df.drop(columns=['month','year'])
    
    return df

df = add_season(df)


In [5]:
# add day of season

def add_dos(df):
    
    df['season_start'] = df['season'].copy() - 1
    df['season_start'] = df['season_start'].copy().astype(str) + '-10-31'
    
    df['DayOfSeason'] = pd.to_datetime(df['date']) - pd.to_datetime(df['season_start'])
    df['DayOfSeason'] = df['DayOfSeason'].dt.days.astype(int)
    
    return df

df = add_dos(df)

print(df.DayOfSeason.min())
print(df.DayOfSeason.max())


6
159


## Features

Time span types
- EWA season
- RA season
- EWA last 5 seasons

Box score feats
- offensive rating
- defensive rating
- pct of points from FTs
- pct of points from 2Pts
- pct of points form 3Pts
- offensive rebound rate
- defensive rebound rate
- pace
- turnover rate
- win pct

*Both for team and opponent



In [6]:
# game ids
def add_ids(df):
    df['wn_copy'] = df['winning_name'].str.replace('\n\t\t\t','').str.replace(' ','').str.lower()
    df['ln_copy'] = df['losing_name'].str.replace('\n\t\t\t','').str.replace(' ','').str.lower()
    
    df['wn_copy2'] = df['wn_copy'].str[:3]
    df['wn_copy3'] = df['wn_copy'].str[-3:]
    
    df['ln_copy2'] = df['ln_copy'].str[:3]
    df['ln_copy3'] = df['ln_copy'].str[-3:]
    
    df['game_id'] = df['date'].dt.strftime('%m%d%y') + df['wn_copy2'] + df['wn_copy3'] + df['ln_copy2']  + df['ln_copy3']
    
#     old = len(df)
    df = df.drop_duplicates(subset=['game_id'])
#     new = len(df)
    # 53184
    
#     print("Successfully dropped {} duplicate box scores".format(old-new))
    
    df = df.drop(columns=['wn_copy2','ln_copy2','wn_copy3','ln_copy3'])
    
    df['team1_id'] = df['season'].astype(str)+df['wn_copy']
    df['team2_id'] = df['season'].astype(str)+df['ln_copy']
    
    df['alt1_id'] = df['wn_copy']
    df['alt2_id'] = df['ln_copy']
    
    df2 = df.copy()
    
    df = df.drop(columns=['team1_id'])
    df2 = df2.drop(columns=['team2_id'])
    df = df.drop(columns=['alt1_id'])
    df2 = df2.drop(columns=['alt2_id'])
    
    df = df.rename(columns={'team2_id':'team_id','alt2_id':'alt_id'})
    df2 = df2.rename(columns={'team1_id':'team_id','alt1_id':'alt_id'})
    
    df = pd.concat([df,df2],axis=0)
    
    df['home_id'] = np.where(df['winner']=='Home', df['season'].astype(str)+df['wn_copy'], df['season'].astype(str)+df['ln_copy'])
    df['away_id'] = np.where(df['winner']=='Away', df['season'].astype(str)+df['wn_copy'], df['season'].astype(str)+df['ln_copy'])
    
    df['win_id'] = df['season'].astype(str)+df['wn_copy']
    df['lose_id'] = df['season'].astype(str)+df['ln_copy']
    
    df = df.drop(columns=['wn_copy','ln_copy'])
    
    print(len(df))
    #106,368
    
    return df

df = add_ids(df)

df[['home_id','away_id','alt_id']].head()


106368


,home_id,away_id,alt_id
0,2011airforce,2011colorado-coloradosprings,colorado-coloradosprings
1,2011airforce,2011coloradocollege,airforce
2,2011airforce,2011tennesseestate,tennesseestate
3,2011wofford,2011airforce,wofford
4,2011airforce,2011calstatenorthridge,calstatenorthridge


In [7]:
# need team-specific ratings leading up to game
# opponent ratings will be harder
home_games = df.loc[df['home_id']==df['team_id']]
away_games = df.loc[df['away_id']==df['team_id']]

del df
gc.collect()

83

In [8]:
# change column names
old_cols = list(home_games)
home_cols = []

for oc in old_cols:
    if 'away_' in oc:
        hc = oc.replace('away_','opp_')
    elif 'home_' in oc:
        hc = oc.replace('home_','')
    else:
        hc = oc
    home_cols.append(hc)
    
# opposite for away_cols
away_cols = []

for oc in old_cols:
    if 'home_' in oc:
        ac = oc.replace('home_','opp_')
    elif 'away_' in oc:
        ac = oc.replace('away_','')
    else:
        ac = oc
    away_cols.append(ac)
    
home_games.columns=home_cols
away_games.columns=away_cols

col_order = list(away_games)
home_games = home_games[col_order]

df = pd.concat([home_games,away_games], axis=0)

# already have this column
df = df.drop(columns=['team_id'])

del home_games
del away_games
gc.collect()


14

In [9]:
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['season','alt_id','date'], ascending=[True,True,True])
df = df.reset_index(drop=True)


In [10]:
# cols ready for running averages
dcols = ['offensive_rating','defensive_rating','opp_offensive_rating','opp_defensive_rating','pace']

# cols needed for feature creation
ncols = ['free_throws','two_point_field_goals','three_point_field_goals',
         'offensive_rebounds','defensive_rebounds','opp_offensive_rebounds','opp_defensive_rebounds',
         'turnovers',
         'wins','losses']

# cols necessary to keep 
nncols = ['date','DayOfSeason','location','id','alt_id','opp_id','game_id','season','win_id','lose_id','points','opp_points']

cols = nncols+dcols+ncols

old_num_cols = len(list(df))
df = df[cols]
new_num_cols = len(list(df))

print("Dropped {} unnecessary columns".format(old_num_cols-new_num_cols))


Dropped 66 unnecessary columns


In [11]:
df.head()

,date,DayOfSeason,location,id,alt_id,opp_id,game_id,season,win_id,lose_id,...,free_throws,two_point_field_goals,three_point_field_goals,offensive_rebounds,defensive_rebounds,opp_offensive_rebounds,opp_defensive_rebounds,turnovers,wins,losses
0,2010-11-14,14,"Clune Arena , Colorado Springs, Colorado",2011airforce,airforce,2011colorado-coloradosprings,111410airrcecolngs,2011,2011airforce,2011colorado-coloradosprings,...,14,14.0,8.0,12,25,11,22,12,1,0
1,2010-11-17,17,"Clune Arena , Colorado Springs, Colorado",2011airforce,airforce,2011coloradocollege,111710colegeairrce,2011,2011coloradocollege,2011airforce,...,16,16.0,3.0,5,30,10,26,16,1,1
2,2010-11-20,20,"Clune Arena , Colorado Springs, Colorado",2011airforce,airforce,2011tennesseestate,112010airrcetenate,2011,2011airforce,2011tennesseestate,...,19,25.0,6.0,7,24,13,15,16,2,1
3,2010-11-24,24,"Benjamin Johnson Arena, Spartanburg, South Car...",2011airforce,airforce,2011wofford,112410airrcewoford,2011,2011airforce,2011wofford,...,18,18.0,6.0,2,21,21,24,15,3,1
4,2010-12-02,32,"Clune Arena , Colorado Springs, Colorado",2011airforce,airforce,2011calstatenorthridge,120210airrcecaldge,2011,2011airforce,2011calstatenorthridge,...,22,13.0,8.0,3,24,21,26,11,4,1


In [12]:
# get home, away, or neutral
# will merge in later
home_arenas = df.groupby(['id'])['location'].apply(pd.Series.mode).reset_index()
home_arenas = home_arenas.drop(columns=['level_1'])
home_arenas

,id,location
0,2011airforce,"Clune Arena , Colorado Springs, Colorado"
1,2011akron,"James A. Rhodes Arena, Akron, Ohio"
2,2011alabama,"Coleman Coliseum , Tuscaloosa, Alabama"
3,2011alabama-birmingham,"Bartow Arena, Birmingham, Alabama"
4,2011alabamaa&m,"Elmore Health Science Building , Normal, Alabama"
5,2011alabamastate,"ASU Acadome, Montgomery, Alabama"
6,2011albany(ny),"SEFCU Arena, Albany, New York"
7,2011albanystate(ga),"ASU Acadome, Montgomery, Alabama"
8,2011alcornstate,"Davey L. Whitney Complex, Alcorn State, Missis..."
9,2011allen,"McLendon-McDougald Gym, Durham, North Carolina"


In [13]:
# let's do season by season first
# easy first
gb = df.groupby(['season','alt_id'])['offensive_rating','defensive_rating','pace'].mean().reset_index()

gb[['ORtg','DRtg','Pace']] = gb.groupby(['alt_id'])['offensive_rating','defensive_rating','pace'].shift()

gb = gb.dropna()

def team_ewm(team, span=5, alpha=0.85):
    feature_ewm = team.rolling(window=span, min_periods=1).mean()[:span]
    rest = team[span:]
    return pd.concat([feature_ewm, rest]).ewm(alpha=alpha, adjust=False).mean()


gb[['5y_ORtg','5y_DRtg','5y_Pace']] = gb.groupby(['alt_id'])['ORtg','DRtg','Pace'].transform(team_ewm)

gb['id'] = gb['season'].astype(str)+gb['alt_id']

gb = gb[['id','5y_ORtg','5y_DRtg','5y_Pace']]

gb.head()


,id,5y_ORtg,5y_DRtg,5y_Pace
607,2012airforce,101.381250,101.918750,63.278125
608,2012akron,100.638889,94.333333,68.527778
609,2012alabama,99.029730,88.462162,67.016216
610,2012alabama-birmingham,103.645161,94.329032,64.551613
611,2012alabamaa&m,87.492857,90.900000,73.457143


In [14]:
# now we have 3/10 features. rest will require extra step

gb2 = df.groupby(['season','alt_id'])['free_throws','two_point_field_goals','three_point_field_goals','offensive_rebounds','defensive_rebounds','opp_offensive_rebounds','opp_defensive_rebounds','turnovers','wins','losses','pace'].sum().reset_index()

gb2[['fts','2pt','3pt','oreb','dreb','oor','odr','tos','w','l','pace']] = gb2.groupby(['alt_id'])['free_throws','two_point_field_goals','three_point_field_goals','offensive_rebounds','defensive_rebounds','opp_offensive_rebounds','opp_defensive_rebounds','turnovers','wins','losses','pace'].shift()

gb2 = gb2.dropna()

# extra step
gb2['dra'] = gb2['dreb'] + gb2['oor']

# offensive rebounds available
gb2['ora'] = gb2['oreb'] + gb2['odr']

#off/def rebound percentage

gb2['DRebPct'] = gb2['dreb']/gb2['dra']
gb2['ORebPct'] = gb2['oreb']/gb2['ora']
gb2['_points'] = gb2['fts'] + 2*gb2['2pt'] + 3*gb2['3pt']
gb2['FTPct'] = gb2['fts']/gb2['_points']
gb2['2ptPct'] = gb2['2pt']/gb2['_points']
gb2['3ptPct'] = gb2['3pt']/gb2['_points']
gb2['TO%'] = gb2['tos']/gb2['pace']
gb2['gp'] = gb2['w'] + gb2['l']
gb2['W%'] = gb2['w']/gb2['gp']

def team_ewm(team, span=5, alpha=0.85):
    feature_ewm = team.rolling(window=span, min_periods=1).mean()[:span]
    rest = team[span:]
    return pd.concat([feature_ewm, rest]).ewm(alpha=alpha, adjust=False).mean()


gb2[['5y_DReb','5y_OReb','5y_FTPct','5y_2PT','5y_3PT','5y_TO','5y_Win']] = gb2.groupby(['alt_id'])['DRebPct','ORebPct','FTPct','2ptPct','3ptPct','TO%','W%'].transform(team_ewm)

gb2['id'] = gb2['season'].astype(str)+gb2['alt_id']

gb2 = gb2[['id','5y_DReb','5y_OReb','5y_FTPct','5y_2PT','5y_3PT','5y_TO','5y_Win']]

gb2.head()


,id,5y_DReb,5y_OReb,5y_FTPct,5y_2PT,5y_3PT,5y_TO,5y_Win
607,2012airforce,0.667954,0.178824,0.201543,0.245902,0.102218,0.185194,0.583333
608,2012akron,0.691189,0.263473,0.181062,0.241284,0.112124,0.173085,0.612613
609,2012alabama,0.687447,0.354839,0.186413,0.314598,0.061464,0.204468,0.644381
610,2012alabama-birmingham,0.715464,0.313305,0.173667,0.243983,0.112789,0.185398,0.750000
611,2012alabamaa&m,0.682203,0.289340,0.230642,0.280973,0.069137,0.211494,0.475369


In [15]:
df = df.reset_index(drop=True)
df.head()

,date,DayOfSeason,location,id,alt_id,opp_id,game_id,season,win_id,lose_id,...,free_throws,two_point_field_goals,three_point_field_goals,offensive_rebounds,defensive_rebounds,opp_offensive_rebounds,opp_defensive_rebounds,turnovers,wins,losses
0,2010-11-14,14,"Clune Arena , Colorado Springs, Colorado",2011airforce,airforce,2011colorado-coloradosprings,111410airrcecolngs,2011,2011airforce,2011colorado-coloradosprings,...,14,14.0,8.0,12,25,11,22,12,1,0
1,2010-11-17,17,"Clune Arena , Colorado Springs, Colorado",2011airforce,airforce,2011coloradocollege,111710colegeairrce,2011,2011coloradocollege,2011airforce,...,16,16.0,3.0,5,30,10,26,16,1,1
2,2010-11-20,20,"Clune Arena , Colorado Springs, Colorado",2011airforce,airforce,2011tennesseestate,112010airrcetenate,2011,2011airforce,2011tennesseestate,...,19,25.0,6.0,7,24,13,15,16,2,1
3,2010-11-24,24,"Benjamin Johnson Arena, Spartanburg, South Car...",2011airforce,airforce,2011wofford,112410airrcewoford,2011,2011airforce,2011wofford,...,18,18.0,6.0,2,21,21,24,15,3,1
4,2010-12-02,32,"Clune Arena , Colorado Springs, Colorado",2011airforce,airforce,2011calstatenorthridge,120210airrcecaldge,2011,2011airforce,2011calstatenorthridge,...,22,13.0,8.0,3,24,21,26,11,4,1


In [16]:
# compute vectors for every team at time of each game
# already sorted by date

# easy first

# season-to-date avg
df[['ORtg','DRtg','Pace']] = df.groupby(['id'])['offensive_rating','defensive_rating','pace'].shift()

# expanding mean
em = df.groupby(['id'])['ORtg','DRtg','Pace'].expanding().mean().reset_index()

if 'level_1' in list(em):
    em = em.drop(columns=['level_1'])

em.head()

# expanding weighted mean

def team_ewm(team, span=50, alpha=0.84):
    feature_ewm = team.rolling(window=span, min_periods=1).mean()[:span]
    rest = team[span:]
    return pd.concat([feature_ewm, rest]).ewm(alpha=alpha, adjust=False).mean()


ewm = df.groupby(['id'])['ORtg','DRtg','Pace'].apply(team_ewm).reset_index()

if 'level_1' in list(ewm):
    ewm = ewm.drop(columns=['level_1'])

ewm.columns=['id','w_ORtg', 'w_DRtg','w_Pace']

gb3 = pd.concat([em, ewm.drop(columns=['id'])], axis=1)

gb3.columns=['id','Sea_ORtg','Sea_DRtg','Sea_Pace','wSea_ORtg','wSea_DRtg','wSea_Pace']

gb3.head()

,id,Sea_ORtg,Sea_DRtg,Sea_Pace,wSea_ORtg,wSea_DRtg,wSea_Pace
0,2011airforce,NaN,NaN,NaN,NaN,NaN,NaN
1,2011airforce,106.500000,85.500000,61.800000,106.500000,85.50000,61.800000
2,2011airforce,93.950000,85.600000,62.000000,95.958000,85.58400,61.968000
3,2011airforce,103.466667,90.866667,65.133333,102.265280,90.02144,64.626880
4,2011airforce,101.925000,90.450000,65.375000,101.979445,90.38143,65.255301


In [17]:
# now let's do the other

# shift up one
df[['fts','2pt','3pt','oreb','dreb','oor','odr','tos','w','l','Pace']] = df.groupby(['id'])['free_throws','two_point_field_goals','three_point_field_goals','offensive_rebounds','defensive_rebounds','opp_offensive_rebounds','opp_defensive_rebounds','turnovers','wins','losses','pace'].shift()


es = df.groupby(['id'])['fts','2pt','3pt','oreb','dreb','oor','odr','tos','w','l','Pace'].expanding().sum().reset_index()


if 'level_1' in list(es):
    es = es.drop(columns=['level_1'])
    
    
# defensive rebounds available
es['dra'] = es['dreb'] + es['oor']

# offensive rebounds available
es['ora'] = es['oreb'] + es['odr']

#off/def rebound percentage

es['DRebPct'] = es['dreb']/es['dra']
es['ORebPct'] = es['oreb']/es['ora']
es['_points'] = es['fts'] + 2*es['2pt'] + 3*es['3pt']
es['FTPct'] = es['fts']/es['_points']
es['2ptPct'] = es['2pt']/es['_points']
es['3ptPct'] = es['3pt']/es['_points']
es['TO%'] = es['tos']/es['Pace']
es['gp'] = es['w'] + es['l']
es['W%'] = es['w']/es['gp']

es.head()



,id,fts,2pt,3pt,oreb,dreb,oor,odr,tos,w,...,ora,DRebPct,ORebPct,_points,FTPct,2ptPct,3ptPct,TO%,gp,W%
0,2011airforce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011airforce,14.0,14.0,8.0,12.0,25.0,11.0,22.0,12.0,1.0,...,34.0,0.694444,0.352941,66.0,0.212121,0.212121,0.121212,0.194175,1.0,1.000000
2,2011airforce,30.0,30.0,11.0,17.0,55.0,21.0,48.0,28.0,2.0,...,65.0,0.723684,0.261538,123.0,0.243902,0.243902,0.089431,0.225806,3.0,0.666667
3,2011airforce,49.0,55.0,17.0,24.0,79.0,34.0,63.0,44.0,4.0,...,87.0,0.699115,0.275862,210.0,0.233333,0.261905,0.080952,0.225179,6.0,0.666667
4,2011airforce,67.0,73.0,23.0,26.0,100.0,55.0,87.0,59.0,7.0,...,113.0,0.645161,0.230088,282.0,0.237589,0.258865,0.081560,0.225621,10.0,0.700000


In [18]:
# weighted sum

def team_ews(team, span=50, alpha=0.84):
    feature_ewm = team.rolling(window=span, min_periods=1).sum()[:span]
    rest = team[span:]
    return pd.concat([feature_ewm, rest]).ewm(alpha=alpha, adjust=False).mean()


ews = df.groupby(['id'])['fts','2pt','3pt','oreb','dreb','oor','odr','tos','w','l','Pace'].apply(team_ews).reset_index()

if 'level_1' in list(ews):
    ews = ews.drop(columns=['level_1'])
    
    
# defensive rebounds available
ews['dra'] = ews['dreb'] + ews['oor']

# offensive rebounds available
ews['ora'] = ews['oreb'] + ews['odr']

#off/def rebound percentage

ews['DRebPct'] = ews['dreb']/ews['dra']
ews['ORebPct'] = ews['oreb']/ews['ora']
ews['_points'] = ews['fts'] + 2*ews['2pt'] + 3*ews['3pt']
ews['FTPct'] = ews['fts']/ews['_points']
ews['2ptPct'] = ews['2pt']/ews['_points']
ews['3ptPct'] = ews['3pt']/ews['_points']
ews['TO%'] = ews['tos']/ews['Pace']
ews['gp'] = ews['w'] + ews['l']
ews['W%'] = ews['w']/ews['gp']

es = es[['DRebPct','ORebPct','FTPct','2ptPct','3ptPct','TO%','W%']]
ews = ews[['DRebPct','ORebPct','FTPct','2ptPct','3ptPct','TO%','W%']]

es.columns= ['Sea_DReb','Sea_OReb','Sea_FT','Sea_2pt','Sea_3pt','Sea_TO','Sea_W']
ews.columns= ['wSea_DReb','wSea_OReb','wSea_FT','wSea_2pt','wSea_3pt','wSea_TO','wSea_W']

gb3 = pd.concat([gb3, es, ews], axis=1)

gb3.head()



,id,Sea_ORtg,Sea_DRtg,Sea_Pace,wSea_ORtg,wSea_DRtg,wSea_Pace,Sea_DReb,Sea_OReb,Sea_FT,...,Sea_3pt,Sea_TO,Sea_W,wSea_DReb,wSea_OReb,wSea_FT,wSea_2pt,wSea_3pt,wSea_TO,wSea_W
0,2011airforce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011airforce,106.500000,85.500000,61.800000,106.500000,85.50000,61.800000,0.694444,0.352941,0.212121,...,0.121212,0.194175,1.000000,0.694444,0.352941,0.212121,0.212121,0.121212,0.194175,1.000000
2,2011airforce,93.950000,85.600000,62.000000,95.958000,85.58400,61.968000,0.723684,0.261538,0.243902,...,0.089431,0.225806,0.666667,0.721264,0.269820,0.240955,0.240955,0.092378,0.223064,0.686567
3,2011airforce,103.466667,90.866667,65.133333,102.265280,90.02144,64.626880,0.699115,0.275862,0.233333,...,0.080952,0.225179,0.666667,0.701441,0.275160,0.234047,0.259943,0.082022,0.224967,0.668227
4,2011airforce,101.925000,90.450000,65.375000,101.979445,90.38143,65.255301,0.645161,0.230088,0.237589,...,0.081560,0.225621,0.700000,0.651650,0.235602,0.237177,0.258991,0.081614,0.225545,0.697003


In [19]:
from functools import reduce

gbs = [gb3, gb2, gb]

feats = reduce(lambda left,right: pd.merge(left,right,on='id', how='left'), gbs)

In [20]:
feats.head()

,id,Sea_ORtg,Sea_DRtg,Sea_Pace,wSea_ORtg,wSea_DRtg,wSea_Pace,Sea_DReb,Sea_OReb,Sea_FT,...,5y_DReb,5y_OReb,5y_FTPct,5y_2PT,5y_3PT,5y_TO,5y_Win,5y_ORtg,5y_DRtg,5y_Pace
0,2011airforce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011airforce,106.500000,85.500000,61.800000,106.500000,85.50000,61.800000,0.694444,0.352941,0.212121,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011airforce,93.950000,85.600000,62.000000,95.958000,85.58400,61.968000,0.723684,0.261538,0.243902,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011airforce,103.466667,90.866667,65.133333,102.265280,90.02144,64.626880,0.699115,0.275862,0.233333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011airforce,101.925000,90.450000,65.375000,101.979445,90.38143,65.255301,0.645161,0.230088,0.237589,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
print(len(list(feats)))
print(len(feats))
feat_cols = list(feats)
feat_cols.remove('id')
feat_cols.insert(0,'home_loc')
feat_cols.insert(0,'location')
feat_cols.insert(0,'DayOfSeason')
# feat_cols.insert(0, 'season') # does not help
feat_cols.insert(0,'points')
feat_cols.insert(0,'game_id')

31
106368


In [22]:
print(len(home_arenas))
home_arenas = home_arenas.drop_duplicates(subset=['id'])
print(len(home_arenas))

home_arenas = home_arenas.rename(columns={'location':'home_loc'})

feats = pd.merge(feats, home_arenas, on=['id','id'],how='left')


6202
5109


In [23]:
print(list(df))

['date', 'DayOfSeason', 'location', 'id', 'alt_id', 'opp_id', 'game_id', 'season', 'win_id', 'lose_id', 'points', 'opp_points', 'offensive_rating', 'defensive_rating', 'opp_offensive_rating', 'opp_defensive_rating', 'pace', 'free_throws', 'two_point_field_goals', 'three_point_field_goals', 'offensive_rebounds', 'defensive_rebounds', 'opp_offensive_rebounds', 'opp_defensive_rebounds', 'turnovers', 'wins', 'losses', 'ORtg', 'DRtg', 'Pace', 'fts', '2pt', '3pt', 'oreb', 'dreb', 'oor', 'odr', 'tos', 'w', 'l']


In [24]:
df = df.drop(columns=['id']) # use feat id
df = pd.concat([df,feats],axis=1)

In [25]:
ids = df[feat_cols]
ids.loc[:,'at_home'] = np.where(ids['location'].copy()==ids['home_loc'].copy(),1,0)
ids = ids.drop(columns=['location','home_loc'])

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [26]:
ids.head()

,game_id,points,DayOfSeason,Sea_ORtg,Sea_DRtg,Sea_Pace,wSea_ORtg,wSea_DRtg,wSea_Pace,Sea_DReb,...,5y_OReb,5y_FTPct,5y_2PT,5y_3PT,5y_TO,5y_Win,5y_ORtg,5y_DRtg,5y_Pace,at_home
0,111410airrcecolngs,66,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,111710colegeairrce,57,17,106.500000,85.500000,61.800000,106.500000,85.50000,61.800000,0.694444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,112010airrcetenate,87,20,93.950000,85.600000,62.000000,95.958000,85.58400,61.968000,0.723684,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,112410airrcewoford,72,24,103.466667,90.866667,65.133333,102.265280,90.02144,64.626880,0.699115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,120210airrcecaldge,72,32,101.925000,90.450000,65.375000,101.979445,90.38143,65.255301,0.645161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [27]:
team1 = ids.drop_duplicates(subset=['game_id'],keep='first')

team2 = ids.drop_duplicates(subset=['game_id'],keep='last')

del ids
gc.collect()

68

In [28]:
train1 = pd.merge(team1, team2, on=['game_id','game_id'], how='left')
train2 = pd.merge(team2, team1, on=['game_id','game_id'], how='left')

train = pd.concat([train1, train2],axis=0)

del train1
del train2
gc.collect()

print(len(train))
train.head()

106368


,game_id,points_x,DayOfSeason_x,Sea_ORtg_x,Sea_DRtg_x,Sea_Pace_x,wSea_ORtg_x,wSea_DRtg_x,wSea_Pace_x,Sea_DReb_x,...,5y_OReb_y,5y_FTPct_y,5y_2PT_y,5y_3PT_y,5y_TO_y,5y_Win_y,5y_ORtg_y,5y_DRtg_y,5y_Pace_y,at_home_y
0,111410airrcecolngs,66,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,111710colegeairrce,57,17,106.500000,85.500000,61.800000,106.500000,85.50000,61.800000,0.694444,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,112010airrcetenate,87,20,93.950000,85.600000,62.000000,95.958000,85.58400,61.968000,0.723684,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,112410airrcewoford,72,24,103.466667,90.866667,65.133333,102.265280,90.02144,64.626880,0.699115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,120210airrcecaldge,72,32,101.925000,90.450000,65.375000,101.979445,90.38143,65.255301,0.645161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [30]:
# neutral sites
train['at_home_x'] = np.where((train['at_home_x'].copy()==0)&(train['at_home_y'].copy()==0), 0.5, train['at_home_x'].copy())

train['target'] = train['points_x'].copy() - train['points_y'].copy()

train = train.drop(columns=['DayOfSeason_y','at_home_y','points_x','points_y','game_id'])



In [31]:
train.head()

,DayOfSeason_x,Sea_ORtg_x,Sea_DRtg_x,Sea_Pace_x,wSea_ORtg_x,wSea_DRtg_x,wSea_Pace_x,Sea_DReb_x,Sea_OReb_x,Sea_FT_x,...,5y_OReb_y,5y_FTPct_y,5y_2PT_y,5y_3PT_y,5y_TO_y,5y_Win_y,5y_ORtg_y,5y_DRtg_y,5y_Pace_y,target
0,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13
1,17,106.500000,85.500000,61.800000,106.500000,85.50000,61.800000,0.694444,0.352941,0.212121,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3
2,20,93.950000,85.600000,62.000000,95.958000,85.58400,61.968000,0.723684,0.261538,0.243902,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
3,24,103.466667,90.866667,65.133333,102.265280,90.02144,64.626880,0.699115,0.275862,0.233333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
4,32,101.925000,90.450000,65.375000,101.979445,90.38143,65.255301,0.645161,0.230088,0.237589,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9


In [32]:
train.to_csv('./output/train.csv',index=None)

In [33]:
train.shape

(106368, 63)